In [1]:
import paddle, paddlenlp
import torch, transformers
import numpy as np
from transformers import MT5Model as PTMT5Model
from transformers import T5Tokenizer as PTT5Tokenizer
from paddlenlp.transformers.mt5 import MT5Model as PDMT5MODEL
from paddlenlp.transformers.mt5 import T5Tokenizer as PDT5Tokenizer
from reprod_log import ReprodLogger


D:\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable


In [6]:
### 对齐tokenizer
text = "Welcome to use paddle and paddlenlp!"
torch_tokenizer = PTT5Tokenizer.from_pretrained("./mt5-large")
paddle_tokenizer = PDT5Tokenizer.from_pretrained("./mt5-large")
torch_inputs = torch_tokenizer(text)
paddle_inputs = paddle_tokenizer(text)
print(torch_inputs)
print(paddle_inputs)


file ./mt5-large\config.json not found


{'input_ids': [259, 29230, 288, 2225, 421, 93575, 305, 421, 93575, 272, 280, 325, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [259, 29230, 288, 2225, 421, 93575, 305, 421, 93575, 272, 280, 325, 309, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [2]:
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien."

reprod_logger = ReprodLogger()

# torch output
import torch
import transformers
from transformers import MT5Model, T5Tokenizer

PREFIX = "C:/Users/QYS/Desktop/"
torch_model = MT5Model.from_pretrained(f"{PREFIX}torch/mt5-large")
torch_tokenizer = T5Tokenizer.from_pretrained("./mt5-large")
torch_model.eval()
# print("111",torch_model.dummy_inputs)

torch_inputs = torch_tokenizer(article, return_tensors="pt")
with torch_tokenizer.as_target_tokenizer():
    labels = torch_tokenizer(summary, return_tensors="pt")
print("input", torch_inputs)
print("labels", labels)
torch_outputs = torch_model(input_ids=torch_inputs["input_ids"], decoder_input_ids=labels["input_ids"])
# print("output", torch_outputs)
torch_logits = torch_outputs.last_hidden_state
torch_array = torch_logits.cpu().detach().numpy()
print("torch_prediction_logits shape:{}".format(torch_array.shape))
print("torch_prediction_logits:{}".format(torch_array))


reprod_logger.add("logits", torch_logits.cpu().detach().numpy())
reprod_logger.save("forward_torch.npy")


# paddle output
import paddle
import paddlenlp
from paddlenlp.transformers.mt5 import MT5Model, T5Tokenizer
import numpy as np

# paddle_model = BertForPretraining.from_pretrained(paddle_model_name)
paddle_model = MT5Model.from_pretrained("mt5-large")
paddle_tokenizer = T5Tokenizer.from_pretrained("./mt5-large")
paddle_model.eval()
# print("111",paddle_model.dummy_inputs)

paddle_inputs = paddle_tokenizer(article)
labels = paddle_tokenizer(summary)

paddle_inputs = {k:paddle.to_tensor([v]) for (k, v) in paddle_inputs.items()}
labels = {k:paddle.to_tensor([v]) for (k, v) in labels.items()}
print("input", paddle_inputs)
print("labels", labels)
paddle_outputs = paddle_model(input_ids=paddle_inputs["input_ids"], decoder_input_ids=labels["input_ids"],return_dict=True)
# print("output", paddle_outputs)
paddle_logits = paddle_outputs.last_hidden_state
paddle_array = paddle_logits.numpy()
print("paddle_prediction_logits shape:{}".format(paddle_array.shape))
print("paddle_prediction_logits:{}".format(paddle_array))


reprod_logger.add("logits", paddle_logits.cpu().detach().numpy())
reprod_logger.save("forward_paddle.npy")



# the output logits should have the same shape
assert torch_array.shape == paddle_array.shape, "the output logits should have the same shape, but got : {} and {} instead".format(torch_array.shape, paddle_array.shape)
# diff = torch_array - paddle_array
# print(np.amax(abs(diff)))
# print(np.mean(abs(diff)))
a = torch.tensor(torch_array).float()
b = torch.tensor(paddle_array).float()
meandif = (a - b).abs().mean()
maxdif = (a - b).abs().max()
print("mean difference:", meandif)
print("max difference:", maxdif)

Some weights of the model checkpoint at C:/Users/QYS/Desktop/torch/mt5-large were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
file ./mt5-large\config.json not found


input {'input_ids': tensor([[ 10969,    443, 209522,    295,    259,   8992,    261,    259,   3648,
           8104,    672,  53764,   2100,   8497,    281,  55984,    278,    260,
              1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
labels {'input_ids': tensor([[ 54620, 191743,    281,  55984,    278,    260,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
torch_prediction_logits shape:(1, 7, 1024)
torch_prediction_logits:[[[ 0.28420416 -0.50179034 -0.04449071 ... -0.1624943  -0.04400842
    0.1341146 ]
  [ 0.12788972 -0.3753653  -0.14871678 ... -0.19723159 -0.03361434
    0.10348243]
  [ 0.30298987 -0.50692534 -0.28211042 ...  0.04345986  0.06924342
   -0.32323718]
  ...
  [ 0.11287715 -0.26105925 -0.42680943 ... -0.03731987 -0.1992596
   -0.4413458 ]
  [ 0.2958529   0.04502446  0.17576706 ...  0.12110859  0.11719192
   -0.15076399]
  [ 0.18357936 -0.164996   -0.08541787 ...  0.04477478 -0.07690276
   -0.22456217]]]

[2021-10-17 16:11:42,141] [    INFO] - Weights from pretrained model not used in MT5Model: ['lm_head.weight']


input {'input_ids': Tensor(shape=[1, 19], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[10969 , 443   , 209522, 295   , 259   , 8992  , 261   , 259   , 3648  , 8104  , 672   , 53764 , 2100  , 8497  , 281   , 55984 , 278   , 260   , 1     ]]), 'token_type_ids': Tensor(shape=[1, 19], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
labels {'input_ids': Tensor(shape=[1, 7], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[54620 , 191743, 281   , 55984 , 278   , 260   , 1     ]]), 'token_type_ids': Tensor(shape=[1, 7], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[0, 0, 0, 0, 0, 0, 0]])}
paddle_prediction_logits shape:(1, 7, 1024)
paddle_prediction_logits:[[[ 0.284203   -0.5017895  -0.0444912  ... -0.1624946  -0.04400791
    0.13411516]
  [ 0.12789306 -0.3753674  -0.14871463 ... -0.19723125 -0.03361408
    0.10348433]
  [ 0.30298844 -0.50692296 -0.28211102 ...  0.04345934  0.06924341
   -0

In [3]:
from reprod_log import ReprodDiffHelper

diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("forward_torch.npy")
paddle_info = diff_helper.load_info("forward_paddle.npy")
diff_helper.compare_info(torch_info, paddle_info)
diff_helper.report(path="forward_diff.log")

AssertionError: k in data2 but not found in data1

In [3]:
import paddle

model_file = "C:/Users/QYS/Desktop/model_state.pdparams"
paddle_state_dict = paddle.load(model_file)
for i in paddle_state_dict.keys():
    print(i, paddle_state_dict[i].shape)

mt5.shared.weight [250112, 1024]
mt5.encoder.embed_tokens.weight [250112, 1024]
mt5.encoder.block.0.layer.0.SelfAttention.q.weight [1024, 1024]
mt5.encoder.block.0.layer.0.SelfAttention.k.weight [1024, 1024]
mt5.encoder.block.0.layer.0.SelfAttention.v.weight [1024, 1024]
mt5.encoder.block.0.layer.0.SelfAttention.o.weight [1024, 1024]
mt5.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight [32, 16]
mt5.encoder.block.0.layer.0.layer_norm.weight [1024]
mt5.encoder.block.0.layer.1.DenseReluDense.wi_0.weight [1024, 2816]
mt5.encoder.block.0.layer.1.DenseReluDense.wi_1.weight [1024, 2816]
mt5.encoder.block.0.layer.1.DenseReluDense.wo.weight [2816, 1024]
mt5.encoder.block.0.layer.1.layer_norm.weight [1024]
mt5.encoder.block.1.layer.0.SelfAttention.q.weight [1024, 1024]
mt5.encoder.block.1.layer.0.SelfAttention.k.weight [1024, 1024]
mt5.encoder.block.1.layer.0.SelfAttention.v.weight [1024, 1024]
mt5.encoder.block.1.layer.0.SelfAttention.o.weight [1024, 1024]
mt5.encoder.block.

In [4]:
import torch
model_file = "C:/Users/QYS/Desktop/torch/mt5-large/pytorch_model.bin"
pytorch_state_dict = torch.load(model_file)
for i in pytorch_state_dict.keys():
    print(i, pytorch_state_dict[i].shape)


shared.weight torch.Size([250112, 1024])
encoder.embed_tokens.weight torch.Size([250112, 1024])
encoder.block.0.layer.0.SelfAttention.q.weight torch.Size([1024, 1024])
encoder.block.0.layer.0.SelfAttention.k.weight torch.Size([1024, 1024])
encoder.block.0.layer.0.SelfAttention.v.weight torch.Size([1024, 1024])
encoder.block.0.layer.0.SelfAttention.o.weight torch.Size([1024, 1024])
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight torch.Size([32, 16])
encoder.block.0.layer.0.layer_norm.weight torch.Size([1024])
encoder.block.0.layer.1.DenseReluDense.wi_0.weight torch.Size([2816, 1024])
encoder.block.0.layer.1.DenseReluDense.wi_1.weight torch.Size([2816, 1024])
encoder.block.0.layer.1.DenseReluDense.wo.weight torch.Size([1024, 2816])
encoder.block.0.layer.1.layer_norm.weight torch.Size([1024])
encoder.block.1.layer.0.SelfAttention.q.weight torch.Size([1024, 1024])
encoder.block.1.layer.0.SelfAttention.k.weight torch.Size([1024, 1024])
encoder.block.1.layer.0.SelfAttent